<a href="https://colab.research.google.com/github/Faur4n/smallworld.js/blob/master/audioClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tarfile

In [2]:
!wget 'http://opihi.cs.uvic.ca/sound/genres.tar.gz'

--2021-02-01 18:26:56--  http://opihi.cs.uvic.ca/sound/genres.tar.gz
Resolving opihi.cs.uvic.ca (opihi.cs.uvic.ca)... 142.104.68.135
Connecting to opihi.cs.uvic.ca (opihi.cs.uvic.ca)|142.104.68.135|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1225571541 (1.1G) [application/x-gzip]
Saving to: ‘genres.tar.gz’

genres.tar.gz       100%[===================>]   1.14G  2.08MB/s    in 9m 20s  

2021-02-01 18:36:16 (2.09 MB/s) - ‘genres.tar.gz’ saved [1225571541/1225571541]



In [8]:
dataset_file_name = './genres.tar.gz'
with tarfile.open(dataset_file_name, 'r') as tar_file:
    tar_file.extractall('./genres')

In [9]:
path_to_dataset = "./genres/genres"

In [27]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
from sklearn.model_selection import train_test_split
import csv
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
from keras import layers
from keras.models import Sequential
import warnings
warnings.filterwarnings('ignore')

In [19]:
cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)
    for filename in os.listdir(f'./genres/genres/{g}'):
        songname = f'./genres/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

<Figure size 576x576 with 0 Axes>

In [21]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [30]:
file = open('dataset.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'./genres/genres/{g}'):
        songname = f'./genres/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        rmse = librosa.feature.rms(y=y)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('dataset.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [62]:
data = pd.read_csv('dataset.csv')
data.head()# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)#Encoding the Labels
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)#Scaling the Feature columns
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))#Dividing data into training and Testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

[[-1.33153204 -0.78071653 -0.92813583 ...  0.80767322  1.92866969
  -0.40594754]
 [-1.28157251 -0.75513785 -0.53705433 ... -0.72431331  1.39064139
  -0.818202  ]
 [-1.07414927 -0.63526038 -1.52635494 ... -0.82100149  0.82064344
   0.14644108]
 ...
 [ 0.89609041 -0.26726183  0.59412565 ... -1.10874368 -0.65930886
  -0.9672601 ]
 [-0.94993909 -0.66167885  0.32918408 ...  1.1866065   0.65356702
   0.72200567]
 [ 0.30145605  0.52694153 -0.37495042 ...  0.7060528  -1.53020522
   0.47664613]]


In [33]:
model = Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [34]:
classifier = model.fit(X_train,
                    y_train,
                    epochs=100,
                    batch_size=128)

Epoch 1/100
7/7 [==============================] - 1s 4ms/step - loss: 2.2396 - accuracy: 0.2095
Epoch 2/100
7/7 [==============================] - 0s 3ms/step - loss: 1.9313 - accuracy: 0.3439
Epoch 3/100
7/7 [==============================] - 0s 4ms/step - loss: 1.7164 - accuracy: 0.3810
Epoch 4/100
7/7 [==============================] - 0s 5ms/step - loss: 1.4910 - accuracy: 0.4968
Epoch 5/100
7/7 [==============================] - 0s 5ms/step - loss: 1.3571 - accuracy: 0.5708
Epoch 6/100
7/7 [==============================] - 0s 5ms/step - loss: 1.2007 - accuracy: 0.6117
Epoch 7/100
7/7 [==============================] - 0s 4ms/step - loss: 1.0762 - accuracy: 0.6702
Epoch 8/100
7/7 [==============================] - 0s 4ms/step - loss: 0.9838 - accuracy: 0.6692
Epoch 9/100
7/7 [==============================] - 0s 4ms/step - loss: 0.9107 - accuracy: 0.7168
Epoch 10/100
7/7 [==============================] - 0s 5ms/step - loss: 0.8575 - accuracy: 0.7139
Epoch 11/100
7/7 [===========

In [61]:
filename = "metal.00000.wav"
songname =  f'./genres/genres/metal/{filename}'
y, sr = librosa.load(songname, mono=True, duration=30)
rmse = librosa.feature.rms(y=y)
chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
zcr = librosa.feature.zero_crossing_rate(y)
mfcc = librosa.feature.mfcc(y=y, sr=sr)
to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
for e in mfcc:
  to_append += f' {np.mean(e)}'
to_append += f' {g}'
print(to_append)
X = model.predict()
print(X)


metal.00000.wav 0.48779988288879395 0.11614155024290085 3274.342156991105 2722.3163670049926 6407.227923035991 0.18307976973684212 -52.901485443115234 50.085777282714844 -11.487131118774414 46.34990692138672 6.55136251449585 10.126185417175293 5.481246471405029 8.855128288269043 5.941895961761475 6.6232781410217285 -4.045888900756836 9.771428108215332 -3.1174025535583496 3.6594033241271973 -6.5072832107543945 1.087841510772705 -4.485400676727295 4.725541114807129 -2.7550318241119385 0.5059655904769897 blues


IndexError: ignored

In [64]:
data = pd.read_csv('dataset.csv')
data.head()# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)#Encoding the Labels
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)#Scaling the Feature columns
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))#
res = classifier.model.predict(X)
print(res)

[[9.87673461e-01 5.62448363e-08 2.50606372e-05 ... 1.38593108e-12
  1.31836923e-05 2.91910583e-05]
 [5.89845330e-03 7.90804916e-04 1.10994153e-01 ... 2.55503608e-07
  2.19098001e-06 5.25676769e-05]
 [9.96483326e-01 1.02949730e-06 6.34401105e-04 ... 9.91863369e-10
  2.46093818e-03 5.18858396e-05]
 ...
 [1.23127165e-07 1.38172788e-06 6.13357406e-03 ... 4.30135924e-06
  4.62005955e-05 9.91163731e-01]
 [2.49244048e-09 2.48784204e-06 1.85033205e-05 ... 5.94423385e-04
  2.79457250e-04 9.76572096e-01]
 [3.96576338e-03 1.01796786e-07 1.28060972e-04 ... 7.02302657e-07
  7.68396705e-02 8.86942565e-01]]
